In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from retry import retry
import time

In [2]:
# manually add long lat coords to locations (or use geopy)
# @retry(tries=3, delay=1)
def get_coordinates(address):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(address)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

In [21]:
streets = pd.read_excel('berk_construction_map_streets.xlsx')
start_ints = streets.Start_Intersection
end_ints = streets.End_Intersection

In [27]:
# Example usage
start_coords_list = []
end_coords_list = []
for i,(s,e) in enumerate(list(zip(start_ints,end_ints))):
    print(i,s,e)
    try:
        start_coords = get_coordinates(s + ', California')
        start_coords_list.append(start_coords)
    except:
        start_coords_list.append(None)
    time.sleep(2)
    try:
        end_coords = get_coordinates(e + ', California')
        end_coords_list.append(end_coords)
    except:
        end_coords_list.append(None)
    time.sleep(2)
    print(start_coords,end_coords)
print(end_coords_list)
print(start_coords_list)

0 Channing Way & Sacramento Street Channing Way & McKinley Avenue
(37.8639324, -122.2811585) None
1 Dana Street & Dwight Way Dana Street & Ward Street
None None
2 Durant Avenue & Milvia Street Durant Avenue & Shattuck Avenue
None (37.866695, -122.2671512)
3 Durant Avenue & Shattuck Avenue Durant Avenue & Fulton Street
(37.866695, -122.2671512) None
4 Hopkins Street & San Pablo Avenue Hopkins Street & Sutter Street
(37.866695, -122.2671512) (37.8884197, -122.2722749)
5 Derby Street & Telegraph Avenue Derby Street & College Avenue
(37.86161, -122.2588175) (37.8618959, -122.2535971)
6 Creston Road & Sunset Lane Creston Road & Grizzly Peak Boulevard
None (37.8925986, -122.2564472)
7 Dwight Way & Piedmont Avenue Dwight Way & End
(37.8659722, -122.2514286) None
8 Grizzly Peak Boulevard & Spruce Street Grizzly Peak Boulevard & Marin Avenue
(37.9043805, -122.270361) (37.8969575, -122.261535)
9 Hillegass Avenue & Dwight Way Hillegass Avenue & South City Limit
None None
10 Latham Lane & Miller A

In [28]:
# add lists to columns
streets['Start_Coords'] = start_coords_list
streets['End_Coords'] = end_coords_list
# to excel
streets.to_excel('berk_construction_map_streets.xlsx',index=False)

In [34]:
constr = pd.read_csv('berk_construction_map.csv')
constr = constr.loc[constr.Status=='Construction',:].loc[constr.Name != 'Street Rehabilitation FY 2023/ FY 2024',:]
constr
constr.to_csv('berk_under_construction_map.csv',index=False)

In [13]:
coords = []
for l in constr.Location:
    print(l)
    if 'CA' not in l:
        try:
            coord = get_coordinates(l)  
        except:
            coord = None
    else:
        try:
            get_coordinates(l + ', California')
        except:
            coord = None
    coords.append(coord)

Addison Street to Dreamland/Channing Way (Landscape and Streetscape)
West side of Aquatic Park (Tree Planting)
South west corner of Aquatic Park (Boating and rowing parking lot)
2012 Berkeley Way
At Rose and Hopkins intersection
2527 San Pablo Avenue
1730 Oregon Street, Berkeley, CA 94703
1700 Hopkins St
1700 Hopkins Street, Berkeley, CA 94707
2001 Ashby Avenue
1901 Hearst St
This project will make improvements to the Ohlone Greenway multi-use pathway, from Virginia Gardens to the south to Santa Fe Avenue to the north. It includes the following intersections: Virginia Gardens, Cedar Street, Rose Street, Hopkins Street, Peralta Avenue, Gilman Street, and Santa Fe Avenue.
1900 Block of Hearst Avenue, between Bonita Street and Milvia Street, Berkeley, CA 94704
1500-1900 Blocks of Hearst Avenue, between Sacramento Street and Milvia Street
Various Locations
Allston Way at Strawberry Creek
Hearst Avenue, Scenic Avenue, Euclid Avenue, Le Conte Avenue, Arch Street, CedarStreet, La Loma Avenue,

In [7]:
constr = pd.read_csv('berk_construction_june.csv')
constr = constr.loc[(constr.StatusApril=='Construction') | (constr.Status=='Construction'),:].loc[constr.Name != 'Street Rehabilitation FY 2023/ FY 2024',:]
constr.to_csv('berk_under_construction_map_june.csv',index=False)

In [ ]:
## NOTES: if location is outside berkeley, add a note in data
## if location has multiple locations is a street/highway, choose either center or most berkeley-relevant portions
 
# add color based on construction type

# add images?

In [2]:
valuation_data = pd.read_csv('valuation_test_data.csv')
valuation_data

,Date Issued,Building Type,Construction Type,Address,Valuation
0,1/27/2020,Mixed,New,"2510 CHANNING Way, BERKELEY, CA 94704",10000000
1,1/8/2020,Residential,Addition/Alteration,"1605 1/2 JULIA St, BERKELEY, CA 94703",71800
2,1/13/2020,Residential,Addition/Alteration,"1312 JOSEPHINE St, BERKELEY, CA 94703",100
3,1/28/2020,Residentia,New,"1327 CARLOTTA Ave, BERKELEY, CA 94703",800000


In [4]:
valuation_data['Address'][0]

'2510 CHANNING Way, BERKELEY, CA 94704'

In [6]:
coords = []
for a in valuation_data.Address:
    try:
        coord = get_coordinates(a)
    except:
        coord = None
    coords.append(coord)

In [9]:
valuation_data['Coordinates'] = coords

In [11]:
# valuation_data.to_csv('valuation_test_data.csv',index=False)
valuation_data.to_json('valuation_test_data.json',orient='records')

In [6]:
permit_data = pd.read_csv('src/data/permits2019_20_clean.csv')
permit_data['Sub.Permit.Type'].unique()
# = permit_data.loc[constr.Status=='Construction',:].loc[constr.Name != 'Street Rehabilitation FY 2023/ FY 2024',:]


array(['Alteration', 'Addition/Alteration', 'Addition', 'New',
       'Demolition', 'Sign', nan], dtype=object)

In [7]:
permit_data_2020 = pd.read_csv('src/data/permits2020_21_clean.csv')

In [10]:
permit_data_2020_new = permit_data_2020.loc[permit_data_2020['Work.Type'] == 'New',:].reset_index(drop=True)
permit_data_2020_new = permit_data_2020_new.drop(['StreetNumber','StreetName','StreetType'], axis=1)
permit_data_2020_new.head()

,Permit.Number,Status,Date.Issued,Building.Type,Work.Type,Valuation,Work.Description,Property.Address
0,B2013-05175,Issued,11/28/2016,Residential,New,3350000.0,"** NEW 3 STORY BUILDING, R-1/S-2/A-1 OCCUPANCY...","1499 UNIVERSITY AVE, Berkeley, CA"
1,B2014-01391,NaN,NaN,Residential,New,300000.0,NEW SINGLE FAMILY DWELLING W/ATTACHED ACCESSOR...,"1471 SCENIC AVE, Berkeley, CA"
2,B2014-05752-REV03,Issued,05/05/2021,Mixed Use,New,8030000.0,Deferred Submittal: Shop drawings for Metal Ha...,"1698 UNIVERSITY AVE, Berkeley, CA"
3,B2014-05752-REV04,Issued,08/04/2021,Mixed Use,New,8030000.0,The changes include: added details and plans f...,"1698 UNIVERSITY AVE, Berkeley, CA"
4,B2014-05786,Issued,01/21/2016,Residential,New,5900000.0,"**PHASE II: NEW 36,554 SQFT MIXED USE BLDG. (2...","1808 UNIVERSITY AVE, Berkeley, CA"


In [11]:
permit_data_new = permit_data.loc[permit_data['Sub.Permit.Type'] == 'New',:].reset_index(drop=True)
permit_data_new = permit_data_new.drop(['Permit.Type','Status','Sub.Type','APN','Contractor.s.Name','Contractor.s.Address'], axis=1)
permit_data_new['Property.Address'] = permit_data_new['Property.Address'].apply(lambda x: x.replace('\n',' '))
permit_data_new

,Permit.Number,Date.Issued,Property.Address,Valuation,Work.Description,Sub.Building.Type,Sub.Permit.Type
0,B2018-03893,1/23/2019,"1461 CURTIS St, BERKELEY, CA 94702",88500.0,Construct new 392 SF ADU,Residential,New
1,B2018-03969,1/14/2019,"800 DWIGHT Way, BERKELEY, CA 94710",2900000.0,1/29/19 - Electrical permit issued for lightin...,Commercial,New
2,B2018-03969,1/14/2019,"800 DWIGHT Way, BERKELEY, CA 94710",2900000.0,1/29/19 - Electrical permit issued for lightin...,Commercial,New
3,B2017-02076,1/14/2019,"1310 UNIVERSITY Ave, BERKELEY, CA 94702",32000.0,New 946 square foot canopy over existing secon...,Commercial,New
4,B2018-04041,1/30/2019,"2132 CURTIS St, BERKELEY, CA 94702",112000.0,Add 300 s.f. ADU,Residential,New
...,...,...,...,...,...,...,...
182,B2020-00577,8/13/2020,"2425 GRANT St, BERKELEY, CA 94703",50000.0,Legalization of existing ADU - currently unocc...,Residential,New
183,B2020-01259,8/25/2020,"1814 VIRGINIA St, BERKELEY, CA 94703",92500.0,New 356 SqFt Accessory Dwelling Unit. (See per...,Residential,New
184,B2020-01652,8/3/2020,"2312 TELEGRAPH Ave, BERKELEY, CA 94704",2850.0,"Install .75"" thick acrylic sign into stucco on...",Residential,New
185,B2020-01331,8/17/2020,"1424 STANNAGE Ave, BERKELEY, CA 94702",35000.0,Convert existing detached accessory building i...,Residential,New


In [11]:
# run for each permit data
coords = []
for a in permit_data_2020_new['Property.Address']:
# for a in permit_data_new['Property.Address']:
    try:
        coord = get_coordinates(a)
    except:
        coord = None
    coords.append(coord)
coords

[(37.87064145, -122.2823649664175),
 (37.8815385, -122.26340982601107),
 (37.870834, -122.277673),
 (37.870834, -122.277673),
 (37.87145375, -122.274963),
 (37.87145375, -122.274963),
 (37.8511281, -122.286666),
 (37.8722635, -122.26652514285715),
 (37.8722635, -122.26652514285715),
 (37.8722635, -122.26652514285715),
 (37.8722635, -122.26652514285715),
 (37.8722635, -122.26652514285715),
 (37.8722635, -122.26652514285715),
 (37.8722635, -122.26652514285715),
 (37.8722635, -122.26652514285715),
 (37.8722635, -122.26652514285715),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.873138, -122.267149),
 (37.87232688888889, -122.26808677777778),
 (37.87232688888889, -122.26808677777778),
 (37.87232688888889, -122.26808677777778),
 (37.87232688888889, -122.26808677777778),
 (37.87232688888889, -

In [12]:
permit_data_2020_new['Coordinates'] = coords
# permit_data_new['Coordinates'] = coords

In [14]:
# permit_data_new.to_json('permit_data.json',orient='records')
permit_data_2020_new.to_json('permit_data2020.json',orient='records')

In [3]:
import json
# clean out duplicates from permit_data.json
permit_full = open('permit_data.json')
data = pd.DataFrame(json.load(permit_full)['info'])
data.head()

,Permit Number,Date Issued,Address,Valuation,Work Description,Building Type,Construction Type,Coordinates,Status
0,B2018-03893,1/23/2019,"1461 CURTIS St, BERKELEY, CA 94702",88500.0,Construct new 392 SF ADU,Residential,New,"[37.8773261, -122.2910914086]",None
1,B2018-03969,1/14/2019,"800 DWIGHT Way, BERKELEY, CA 94710",2900000.0,1/29/19 - Electrical permit issued for lightin...,Commercial,New,"[37.8598058687, -122.2956893939]",None
2,B2017-02076,1/14/2019,"1310 UNIVERSITY Ave, BERKELEY, CA 94702",32000.0,New 946 square foot canopy over existing secon...,Commercial,New,"[37.8693555, -122.2862556665]",None
3,B2018-04041,1/30/2019,"2132 CURTIS St, BERKELEY, CA 94702",112000.0,Add 300 s.f. ADU,Residential,New,"[37.86727805, -122.2894114472]",None
4,B2018-04296,1/3/2019,"800 DWIGHT Way B60, BERKELEY, CA 94710",40000.0,Seismic anchoring of a cooling tower and a fil...,Commercial,New,"[37.8597175131, -122.2956661035]",None


In [4]:
len(data['Permit Number'].unique())
len(data['Address'].unique())
len(data.index)

617

In [5]:
data = data.drop_duplicates(subset=['Permit Number']) # first remove duplicate permit numbers...
data = data.drop_duplicates(subset=['Address', 'Valuation']) # then permits with same address valuation = same construction project
len(data.index)

472

In [6]:
# remove projects with valuations <= 0
data = data.loc[data.Valuation > 0]
len(data.index)

441

In [7]:
data.to_json('permit_data_unique.json',orient='records')

In [27]:
import json
# clean out duplicates from permit_data.json
permit_full = open('permit_data.json')
data = pd.DataFrame(json.load(permit_full)['info'])
data.head()

,Permit Number,Date Issued,Address,Valuation,Work Description,Building Type,Construction Type,Coordinates,Status
0,B2018-03893,1/23/2019,"1461 CURTIS St, BERKELEY, CA 94702",88500.0,Construct new 392 SF ADU,Residential,New,"[37.8773261, -122.2910914086]",None
1,B2018-03969,1/14/2019,"800 DWIGHT Way, BERKELEY, CA 94710",2900000.0,1/29/19 - Electrical permit issued for lightin...,Commercial,New,"[37.8598058687, -122.2956893939]",None
2,B2017-02076,1/14/2019,"1310 UNIVERSITY Ave, BERKELEY, CA 94702",32000.0,New 946 square foot canopy over existing secon...,Commercial,New,"[37.8693555, -122.2862556665]",None
3,B2018-04041,1/30/2019,"2132 CURTIS St, BERKELEY, CA 94702",112000.0,Add 300 s.f. ADU,Residential,New,"[37.86727805, -122.2894114472]",None
4,B2018-04296,1/3/2019,"800 DWIGHT Way B60, BERKELEY, CA 94710",40000.0,Seismic anchoring of a cooling tower and a fil...,Commercial,New,"[37.8597175131, -122.2956661035]",None


In [30]:
'sastas'.upper()

'SASTAS'

In [33]:
import re

def remove_zip(address):
    # Use regular expression to check if the last characters are a comma, space, and 5 digits
    if isinstance(address, (str, bytes)):
        address = address.upper()
        match = re.search(r' \d{5}$', address)
        if match:
            # If there is a match, remove the last characters
            return str(address[:-6])  # Remove ", " and 5 digits
        else:
            # If no match, return the original string
            return str(address)
    else:
        return str(address)

In [34]:
# remove zip code from addresses, some don't have zip code
data['Address'] = data['Address'].apply(remove_zip)

In [37]:
# keep first instance of duplicate addresses
print(len(data.index))
data = data.drop_duplicates('Address',keep='first') # dropped 40 dupes
print(len(data.index))
data.to_json('permit_data_unique.json',orient='records')


441
379
